**★（注意）共有フォルダにあるNotebookは読み込み専用です★**

このNotebookをホームディレクトリにコピーして利用するには、以下のセルを実行してください。

In [ ]:
!cp ~/shared/sample-notebook.ipynb ~/sample-notebook-Copy1.ipynb

ホームディレクトリにコピーしたNotebookを開く ⇒ [sample-notebook-Copy1](../sample-notebook-Copy1.ipynb)

# 簡単なPythonコードの実行例

以下は、[Google Colaboratory](https://colab.research.google.com/) の紹介ページにあるサンプルコードのコピーです。

### サンプルコード (1)

値を計算して変数に保存し、結果を出力する短いPythonスクリプトの例です。

In [ ]:
seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

In [ ]:
seconds_in_a_week = 7 * seconds_in_a_day
seconds_in_a_week

### サンプルコード (2)

numpy を使用してランダムデータを生成し、matplotlib を使用して可視化するPythonスクリプトの例です。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

ys = 200 + np.random.randn(100)
x = [x for x in range(len(ys))]

plt.plot(x, ys, '-')
plt.fill_between(x, ys, 195, where=(ys > 195), facecolor='g', alpha=0.6)

plt.title("Sample Visualization")
plt.show()

# GPUを活用する演習教材の例

このNotebookでは、OCSで構築したJupyterHub環境においてGPUを活用した演習が実現可能なことを例示します。

1. [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) のDockerコンテナ実行
2. [PyTorch公式チュートリアル](https://pytorch.org/tutorials/) のNotebook実行
    > (参考) 日本語訳版 <https://yutaroogawa.github.io/pytorch_tutorials_jp/>


## OpenPose のコンテナ利用

構築したGPU環境を利用して [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) を実行してみます。  
OpenPoseは画像、動画から人の体、手、顔、および足のキーポイントを検出するシステムです。

ここでは、すでにあるOpenPoseのDockerイメージを使用し、アプリケーションコンテナとして実行します。

### OpenPoseコンテナの取得と起動

OpenPoseをアプリケーションコンテナとして実行します。

> Dockerイメージの取得には2〜3分程度を要します。

In [ ]:
!sudo docker run -td --gpus all --name openpose \
  -v $HOME/result:/root/result harbor.vcloud.nii.ac.jp/vcp/openpose:handson_20.11

コンテナが起動されたことを確認します。次のセルを実行して `STATUS` が `Up` と表示されていればコンテナの起動に成功しています。

In [ ]:
!sudo docker ps

### OpenPoseの利用 (1) - 画像ファイルの一括処理

OpenPoseのサンプル画像を処理してみます。

まず初めに、処理前の画像ファイルをコンテナ内からコピーします。

In [ ]:
!sudo docker cp openpose:/root/openpose/examples/media $HOME/

OpenPoseで複数の画像ファイルを一括処理します。

In [ ]:
!sudo docker exec -t openpose \
  build/examples/openpose/openpose.bin --display 0 \
  --image_dir examples/media \
  --write_images /root/result --write_images_format jpg

- [ホームディレクトリ](/)
  - 処理前の画像一覧: `media/*.jpg`
  - 処理結果の画像一覧: `result/*.jpg`

### OpenPoseの利用 (2) - 動画の処理

続けて、OpenPoseのサンプル動画を処理してみます。

In [ ]:
!sudo docker exec -t openpose \
  build/examples/openpose/openpose.bin --display 0 \
  --video /root/openpose/examples/media/video.avi \
  --write_video /root/result/result.avi

処理結果の動画ファイル `result/result.avi`
> ダウンロードして、ローカルPC上の動画再生アプリ等で確認できます。

### GrafanaでVCノードのGPU利用状況を確認する

[Grafana](/grafana/d/vcp/vcp-metrics)で、VCノードに搭載されたGPUの利用状況を各メトリクスのグラフ表示から確認することができます。

- GPU Usage per Node
- GPU Memory Usage per Node

## PyTorch公式チュートリアル

PyTorch公式チュートリアルの中から、「画像データセット CIFAR-10 を使ったクラス分類モデルの訓練」を実行してみます。

> (注) 学習データセットの訓練処理に GPU を使わせるためには、コード追加（後述）が必要です。

### PyTorch60分講座 - クラス分類モデルの訓練

[Tutorials](https://pytorch.org/tutorials/index.html) > [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) > [Training a Classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

以下のセルを実行して `cifar10_tutorial.ipynb` をJupyterHubの [ホームディレクトリ](/) にダウンロードします。

In [ ]:
!curl -L https://pytorch.org/tutorials/_downloads/4e865243430a47a00d551ca0579a6f6c/cifar10_tutorial.ipynb \
    -o $HOME/cifar10_tutorial.ipynb

または、以下の手順で ipynb ファイルをダウンロードして利用します。

1. [Training a Classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) ページ冒頭にある **Download Notebook** のリンクから、ローカルPC上に ipynb ファイルをダウンロードします。
2. ダウンロードした ipynb ファイルをJupyterHubの [ホームディレクトリ](/) にアップロードします。

#### GPUを利用するには …

学習データセットの訓練処理に GPU を使わせるためには、終盤にある **Training on GPU** セクションの説明に従ったコード修正が必要です。
ただし、今回はネットワークのサイズが小さいため、CPUで実行した場合と比較して大幅な高速化にはなりません。

**Training on GPU** セクションの冒頭部分にコードを追加します。

新規にセルを追加し、以下の内容をコピー＆ペーストして実行してください。

<div class="alert alert-info">

```
# GPUデバイスを指定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# ネットワーク内の全モジュールのパラメータとバッファをCUDAテンソルに変換
net.to(device)

# 4. Train the network のループ内で、入力データと正解ラベルもGPUに送るようにする
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data  <<<< これを書き換える
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
```
</div>